In [1]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
from typing import Dict
import os
from pydantic import BaseModel

In [2]:
load_dotenv(override=True)

True

In [3]:
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest 
pulling dde5aa3fc5ff... 100% ▕████████████████▏ 2.0 GB                         
pulling 966de95ca8a6... 100% ▕████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕████████████████▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         
pulling 34bb5ab01051... 100% ▕████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [4]:
!ollama pull gemma3

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling aeda25e63ebd... 100% ▕████████████████▏ 3.3 GB                         
pulling e0a42594d802... 100% ▕████████████████▏  358 B                         
pulling dd084c7d92a3... 100% ▕████████████████▏ 8.4 KB                         
pulling 3116c5225075... 100% ▕████████████████▏   77 B                         
pulling b6ae5839783f... 100% ▕████████████████▏  489 B                         
verifying sha256 digest 
writing manifest 
success 


In [5]:
openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-


In [11]:
instructions1 = "You are an artificial intelligence tutor. You write trivias about artificial intelligence."
instructions2 = "You are a humorous, engaging artificial intelligence tutor. You write trivias about artificial intelligence and make jokes about it."

In [ ]:
LLAMA_BASE_URL = 'http://localhost:11434/v1'

llama_client = AsyncOpenAI(base_url=LLAMA_BASE_URL, api_key="ollama")

llama_model = OpenAIChatCompletionsModel(model="llama3.2", openai_client=llama_client)
gemma_model = OpenAIChatCompletionsModel(model="gemma3", openai_client=llama_client)

In [10]:
ai_tutor1 = Agent("Llama AI Tutor", instructions=instructions1, model=llama_model)
ai_tutor2 = Agent("Gemma AI Tutor", instructions=instructions2, model=gemma_model)

In [12]:
description = "Write a trivia about artificial intelligence"

tool1 = ai_tutor1.as_tool(tool_name="ai_tutor1", tool_description=description)
tool2 = ai_tutor2.as_tool(tool_name="ai_tutor2", tool_description=description)

In [13]:
@function_tool
def send_telegram_message(body: str):
    """ Send a message with the given body to all readers. """
    url = f"https://api.telegram.org/bot{telegram_bot_token}/sendMessage"
    payload = {"chat_id": chat_id, "text": body}
    try:
        response = requests.post(url, data=payload)
        if response.status_code == 200:
            print("Notification sent successfully!")
        else:
            print("Failed to send the message:", response.text)
    except Exception as e:
        print("Error sending Telegram message:", e)

In [17]:
check_instructions = "You can check if the given message is true or not. \
You are given a message and you need to check if the given message is true or not."


trivia_checker = Agent(name="Trivia Checker",
                instructions=check_instructions)

checker_tool = checker.as_tool(tool_name="trivia_checker", tool_description="Check if the given trivia is true or not")


In [16]:
trivia_tools = [checker_tool, send_telegram_message]

In [ ]:
instructions = "You are an trivia checker and message sender. You receive the text of trivia to be send. \
    You first use the trivia_checker tool to check whether the given trivia is true or not, then use the send_telegram_message tool to send the message."

trivia_agent = Agent(
    name="Trivia Checker and sender",
    instructions=instructions,
    tools=trivia_tools,
    model="gpt-4o-mini",
    handoff_description="Check if the given trivia message is true or not and send the message."
)

In [19]:
tools = [tool1, tool2]
handoffs = [trivia_agent]

In [21]:
trivia_manager_instructions = "You are a trivia manager. You use the tools given to you to generate trivias. \
    You never generate trivias yourself; you always use the tools. \
    You try all 2 agent tools at least once before choosing the best one. \
    You can use the tools multiple times if you're not satisfied with the results from the first try. \
    You select the single best trivia using your own judgement of which trivia will be most interesting. \
    After selecting the trivia, you handoff to the Trivia Checker and sender agent to check and sen the trivia message."

trivia_manager = Agent(
    name="Trivia Manager",
    instructions=trivia_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini"
)

message = "Generate the trivia and send it as message."

with trace("Trivia generator"):
    result = await Runner.run(trivia_manager, message)